In [1]:
# --- (Regression Version) ---
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib

BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), '..'))
DATA_PATH = os.path.join(BASE_DIR, 'Results', 'Output', 'cleaned_student_data.csv')
OUT_DIR = os.path.join(BASE_DIR, 'Results', 'Output', 'ModelResults')
os.makedirs(OUT_DIR, exist_ok=True)

# Load data
data = pd.read_csv("/content/cleaned_student_data.csv")

# Split features and target
TARGET = 'G3'
X = data.drop(columns=[TARGET])
y = data[TARGET]

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale for models that need normalization
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [2]:
from sklearn.svm import SVR

param_grid = {'C':[0.1,1,10], 'kernel':['linear','rbf'], 'gamma':['scale','auto']}
grid = GridSearchCV(SVR(), param_grid, cv=5, scoring='r2', n_jobs=-1)
grid.fit(X_train_scaled, y_train)

best_model = grid.best_estimator_
y_pred = best_model.predict(X_test_scaled)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("Best Params:", grid.best_params_)
print(f"MAE:{mae:.3f}, RMSE:{rmse:.3f}, R²:{r2:.3f}")

pd.DataFrame([{'Model':'SVR','MAE':mae,'MSE':mse,'RMSE':rmse,'R2':r2}])\
  .to_csv(os.path.join(OUT_DIR,'svr_results.csv'),index=False)

joblib.dump(best_model, os.path.join(OUT_DIR,'svr_best.joblib'))


Best Params: {'C': 1, 'gamma': 'scale', 'kernel': 'rbf'}
MAE:3.871, RMSE:4.944, R²:0.100


['/Results/Output/ModelResults/svr_best.joblib']